# IBM Applied Data Science Capstone Course by Coursera
### Week 3 Part 1, 2 and 3
- Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.
- Get the geographical coordinates of the neighborhoods in Toronto.
- Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).
***
### 1. Import libraries

## Greetings!

Welcome to my new Jupyter notebook, which I am launching to support me as I work through my ninth class in the Coursera Professional Certificate in Data Science Program. I will use this to experiment, to tweak, and ultimately to complete the capstone. Stay Tuned!


In [21]:
# Python says Hello, world!

print('Hello, world!')

Hello, world!


In [22]:
# Now I'll say: Hello Capstone Project Course!
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Note:
I cannot figure how to get my code to run without some including some things that seem superfluous and/or redundant: please indulge!

In [23]:
#Import libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

## 1. Import data from Wikipedia page and put into dataframe.

In [24]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Request page
response = requests.get(url)

### ...Using BeautifulSoup

In [25]:
#Do the soup, find table, isolate rows
soup = BeautifulSoup(response.text)
table = soup.find('tbody')
tablerows = table.find_all('tr')[1:]

In [26]:
#Define df columns 
cols = ['PostCode', 'Borough', 'Neighbourhood'] 
#Initialize df
df = pd.DataFrame(columns=cols)

In [27]:
#Add rows to df
for i in range(len(tablerows)):
    df.loc[i] = [td.text.strip() for td in tablerows[i].find_all('td')]

In [28]:
#Check
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [29]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df.drop(df[df.Borough == 'Not assigned'].index)

## 2. Drop all Boroughs listed as "Not Assigned" and keep the rest.

In [30]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data', sep=',')

In [31]:
df.head(10)

,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [32]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns 
#will be Queen's Park.

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

## 3. Group Boroughs into Neighborhoods based on Postal Code.

In [33]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows 
#will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

df = df.groupby(['PostCode', 'Borough'])['Neighbourhood'].apply(list).to_frame().reset_index()

#Remove square brackets
df['Neighbourhood'] = df['Neighbourhood'].str.join(', ')

### Make sure the 'new' dataframe's shape is the same as that described by the question.

In [34]:
#Final result
df.shape

(103, 3)

## 4. 'Print' the new, improved dataframe.

In [35]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data', sep=',')

In [36]:
df_m = df.merge(geodata, left_on='PostCode', right_on='Postal Code', how='left')

In [37]:
df_m

,PostCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


## 5. Install and import Folium

In [38]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [39]:
!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium

print('Folium installed and imported!')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.

Folium installed and imported!


In [20]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

FileNotFoundError: File b'Geospatial_Coordinates.csv' does not exist